In [1]:
# Import packages:

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# Scrape raw text from the link:

link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
txt = requests.get(link).text
raw = BeautifulSoup(txt, 'lxml')
content = raw.find('div', class_='mw-parser-output')
table = content.table.tbody

In [13]:
# Find relevant information and construct table:

trs = table.find_all('tr')
tds = []
for tr in trs:
    tds.append(tr.find_all('td'))
postcodes = []
boroughs = []
neighborhoods = []
for td in tds[1:]:
    postcodes.append(td[0].text)
    boroughs.append(td[1].text)
    neighborhoods.append(td[2].text.strip('\n'))
df = pd.DataFrame({'PostalCode': postcodes, 'Borough': boroughs, 'Neighborhood': neighborhoods})
df_valid = df[(df['Borough'] != 'Not assigned')].reset_index(drop = True)
df_valid['Neighborhood'] = np.where(df_valid['Neighborhood'] == 'Not assigned', df_valid['Borough'], \
                                    df_valid['Neighborhood'])
df_grouped = df_valid.groupby(by = ['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x))\
                     .reset_index()
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
# Dimensions:

df_grouped.shape

(103, 3)

In [16]:
geo = pd.read_csv('Geospatial_Coordinates.csv').rename(columns = {'Postal Code': 'PostalCode'})
df_final = df_grouped.merge(geo, how = 'left', on = 'PostalCode')
df_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
# Dimensions:

df_final.shape

(103, 5)